In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

def compute_optical_flow(prev_frame, next_frame):
    """Computes dense optical flow using the Farneback method."""
    flow = cv2.calcOpticalFlowFarneback(
        cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY),
        cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY),
        None, 0.5, 3, 15, 3, 5, 1.2, 0
    )
    return flow

def compute_spatial_histograms(flow, num_mag_bins=50, num_angle_bins=72, grid_size=(4,4)):
    """Computes HOF histograms for spatially divided grid regions."""
    h, w, _ = flow.shape
    grid_h, grid_w = h // grid_size[0], w // grid_size[1]
    
    hof_features = []
    
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            region = flow[i * grid_h: (i + 1) * grid_h, j * grid_w: (j + 1) * grid_w]
            magnitude, angle = cv2.cartToPolar(region[..., 0], region[..., 1])
            angle = np.degrees(angle) % 360  # Normalize angle to [0, 360] degrees
            
            mag_hist, _ = np.histogram(magnitude, bins=num_mag_bins, range=(0, np.max(magnitude)))
            angle_hist, _ = np.histogram(angle, bins=num_angle_bins, range=(0, 360))
            
            mag_hist = mag_hist.astype(np.float32) / np.sum(mag_hist) if np.sum(mag_hist) > 0 else mag_hist
            angle_hist = angle_hist.astype(np.float32) / np.sum(angle_hist) if np.sum(angle_hist) > 0 else angle_hist
            
            hof_features.extend(mag_hist)
            hof_features.extend(angle_hist)
    
    return np.array(hof_features)

def process_video_frames(frame_folder, num_mag_bins=50, num_angle_bins=72, grid_size=(4,4)):
    """Computes fine-grained HOF for a sequence of frames."""
    frame_files = sorted(os.listdir(frame_folder))
    num_frames = len(frame_files)

    if num_frames < 2:
        print(f"Skipping {frame_folder}, not enough frames.")
        return None

    combined_features = []
    
    for i in range(num_frames - 1):
        frame1 = cv2.imread(os.path.join(frame_folder, frame_files[i]))
        frame2 = cv2.imread(os.path.join(frame_folder, frame_files[i + 1]))
        
        if frame1 is None or frame2 is None:
            continue

        flow = compute_optical_flow(frame1, frame2)
        hof_features = compute_spatial_histograms(flow, num_mag_bins, num_angle_bins, grid_size)
        combined_features.append(hof_features)
    
    return np.array(combined_features)

def preprocess_dataset(dataset_path, output_path, num_mag_bins=30, num_angle_bins=36, grid_size=(4,4)):
    """Processes dataset and saves HOF features."""
    os.makedirs(output_path, exist_ok=True)
    
    for sample_folder in tqdm(os.listdir(dataset_path)):
        sample_path = os.path.join(dataset_path, sample_folder)
        
        if not os.path.isdir(sample_path):
            continue
        
        combined_features = process_video_frames(sample_path, num_mag_bins, num_angle_bins, grid_size)
        
        if combined_features is not None:
            output_file = os.path.join(output_path, f"{sample_folder}_hof.npy")
            np.save(output_file, combined_features)

# Paths
dataset_path = "../dataset_100_3_points"  # Update with actual path
output_path = "../precomputed_hof_100"

# Run preprocessing
preprocess_dataset(dataset_path, output_path)
print("Fine-grained HOF preprocessing completed!")


The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


100%|██████████| 320/320 [03:10<00:00,  1.68it/s]

Fine-grained HOF preprocessing completed!


# Augmentation

In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from scipy.ndimage import gaussian_filter1d
import random

def apply_augmentations(frame):
    """Applies random augmentations to the given frame."""
    if random.random() < 0.5:
        frame = cv2.flip(frame, 1)  # Horizontal flip
    
    if random.random() < 0.5:
        factor = random.uniform(0.8, 1.2)  # Random brightness
        frame = np.clip(frame * factor, 0, 255).astype(np.uint8)
    
    if random.random() < 0.5:
        noise = np.random.normal(0, 10, frame.shape).astype(np.uint8)  # Gaussian noise
        frame = np.clip(frame + noise, 0, 255).astype(np.uint8)
    
    if random.random() < 0.5:
        angle = random.uniform(-10, 10)  # Random rotation
        h, w = frame.shape[:2]
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
        frame = cv2.warpAffine(frame, M, (w, h))
    
    return frame

def compute_optical_flow(prev_frame, next_frame):
    gray1 = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    return flow

def compute_spatial_histograms(flow, num_mag_bins=30, num_angle_bins=36, grid_size=(4,4), smooth_sigma=1.0):
    h, w, _ = flow.shape
    grid_h, grid_w = h // grid_size[0], w // grid_size[1]
    hof_features = []
    
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            region = flow[i * grid_h: (i + 1) * grid_h, j * grid_w: (j + 1) * grid_w]
            magnitude, angle = cv2.cartToPolar(region[..., 0], region[..., 1])
            angle = np.degrees(angle) % 360  

            mag_hist, _ = np.histogram(magnitude, bins=num_mag_bins, range=(0, np.max(magnitude) if np.max(magnitude) > 0 else 1))
            angle_hist, _ = np.histogram(angle, bins=num_angle_bins, range=(0, 360))

            mag_hist = mag_hist / (np.sum(mag_hist) + 1e-6)
            angle_hist = angle_hist / (np.sum(angle_hist) + 1e-6)

            mag_hist = gaussian_filter1d(mag_hist, sigma=smooth_sigma)
            angle_hist = gaussian_filter1d(angle_hist, sigma=smooth_sigma)

            hof_features.extend(mag_hist)
            hof_features.extend(angle_hist)
    
    return np.array(hof_features, dtype=np.float32)

def process_video_frames(frame_folder, num_aug=2):
    frame_files = sorted(os.listdir(frame_folder))
    num_frames = len(frame_files)
    
    if num_frames < 2:
        return None, None, None

    original_features = []
    augmented_features_1 = []
    augmented_features_2 = []
    
    for i in range(num_frames - 1):
        frame1 = cv2.imread(os.path.join(frame_folder, frame_files[i]))
        frame2 = cv2.imread(os.path.join(frame_folder, frame_files[i + 1]))
        
        if frame1 is None or frame2 is None:
            continue

        # Original optical flow
        flow = compute_optical_flow(frame1, frame2)
        hof_features = compute_spatial_histograms(flow)
        original_features.append(hof_features)

        # Augmentation 1
        aug_frame1 = apply_augmentations(frame1.copy())
        aug_frame2 = apply_augmentations(frame2.copy())
        aug_flow1 = compute_optical_flow(aug_frame1, aug_frame2)
        aug_hof_features1 = compute_spatial_histograms(aug_flow1)
        augmented_features_1.append(aug_hof_features1)

        # Augmentation 2
        aug_frame1 = apply_augmentations(frame1.copy())
        aug_frame2 = apply_augmentations(frame2.copy())
        aug_flow2 = compute_optical_flow(aug_frame1, aug_frame2)
        aug_hof_features2 = compute_spatial_histograms(aug_flow2)
        augmented_features_2.append(aug_hof_features2)

    return np.array(original_features), np.array(augmented_features_1), np.array(augmented_features_2)

def preprocess_dataset(dataset_path, output_path):
    os.makedirs(output_path, exist_ok=True)
    
    for sample_folder in tqdm(os.listdir(dataset_path), desc="Processing videos"):
        sample_path = os.path.join(dataset_path, sample_folder)
        if not os.path.isdir(sample_path):
            continue

        original, aug1, aug2 = process_video_frames(sample_path, num_aug=2)
        
        if original is not None and len(original) > 0:
            np.save(os.path.join(output_path, f"{sample_folder}_hof.npy"), original)
            np.save(os.path.join(output_path, f"{sample_folder}_aug1_hof.npy"), aug1)
            np.save(os.path.join(output_path, f"{sample_folder}_aug2_hof.npy"), aug2)

# Paths
dataset_path = "../dataset_100_3_points"
output_path = "../precomputed_hof_augmented_100"

# Run preprocessing
preprocess_dataset(dataset_path, output_path)
print("Data augmentation and HOF preprocessing completed!")


Processing videos: 100%|██████████| 320/320 [09:50<00:00,  1.85s/it]

Data augmentation and HOF preprocessing completed!


In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from scipy.ndimage import gaussian_filter1d
import random

def apply_augmentations(frame):
    """Applies random augmentations to the given frame."""
    if random.random() < 0.5:
        frame = cv2.flip(frame, 1)  # Horizontal flip
    
    if random.random() < 0.5:
        factor = random.uniform(0.8, 1.2)  # Random brightness
        frame = np.clip(frame * factor, 0, 255).astype(np.uint8)
    
    if random.random() < 0.5:
        noise = np.random.normal(0, 10, frame.shape).astype(np.uint8)  # Gaussian noise
        frame = np.clip(frame + noise, 0, 255).astype(np.uint8)
    
    if random.random() < 0.5:
        angle = random.uniform(-10, 10)  # Random rotation
        h, w = frame.shape[:2]
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
        frame = cv2.warpAffine(frame, M, (w, h))
    
    return frame

def compute_optical_flow(prev_frame, next_frame):
    gray1 = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    return flow

def compute_spatial_histograms(flow, num_mag_bins=30, num_angle_bins=36, grid_size=(4,4), smooth_sigma=1.0):
    h, w, _ = flow.shape
    grid_h, grid_w = h // grid_size[0], w // grid_size[1]
    hof_features = []
    
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            region = flow[i * grid_h: (i + 1) * grid_h, j * grid_w: (j + 1) * grid_w]
            magnitude, angle = cv2.cartToPolar(region[..., 0], region[..., 1])
            angle = np.degrees(angle) % 360  

            mag_hist, _ = np.histogram(magnitude, bins=num_mag_bins, range=(0, np.max(magnitude) if np.max(magnitude) > 0 else 1))
            angle_hist, _ = np.histogram(angle, bins=num_angle_bins, range=(0, 360))

            mag_hist = mag_hist / (np.sum(mag_hist) + 1e-6)
            angle_hist = angle_hist / (np.sum(angle_hist) + 1e-6)

            mag_hist = gaussian_filter1d(mag_hist, sigma=smooth_sigma)
            angle_hist = gaussian_filter1d(angle_hist, sigma=smooth_sigma)

            hof_features.extend(mag_hist)
            hof_features.extend(angle_hist)
    
    return np.array(hof_features, dtype=np.float32)

def process_video_frames(frame_folder, num_aug=4):
    frame_files = sorted(os.listdir(frame_folder))
    num_frames = len(frame_files)
    
    if num_frames < 2:
        return None, None

    original_features = []
    augmented_features = [[] for _ in range(num_aug)]
    
    for i in range(num_frames - 1):
        frame1 = cv2.imread(os.path.join(frame_folder, frame_files[i]))
        frame2 = cv2.imread(os.path.join(frame_folder, frame_files[i + 1]))
        
        if frame1 is None or frame2 is None:
            continue

        # Original optical flow
        flow = compute_optical_flow(frame1, frame2)
        hof_features = compute_spatial_histograms(flow)
        original_features.append(hof_features)

        # Generate multiple augmentations
        for j in range(num_aug):
            aug_frame1 = apply_augmentations(frame1.copy())
            aug_frame2 = apply_augmentations(frame2.copy())
            aug_flow = compute_optical_flow(aug_frame1, aug_frame2)
            aug_hof_features = compute_spatial_histograms(aug_flow)
            augmented_features[j].append(aug_hof_features)

    return np.array(original_features), [np.array(aug) for aug in augmented_features]

def preprocess_dataset(dataset_path, output_path, num_aug=4):
    os.makedirs(output_path, exist_ok=True)
    
    for sample_folder in tqdm(os.listdir(dataset_path), desc="Processing videos"):
        sample_path = os.path.join(dataset_path, sample_folder)
        if not os.path.isdir(sample_path):
            continue

        original, augmentations = process_video_frames(sample_path, num_aug=num_aug)
        
        if original is not None and len(original) > 0:
            np.save(os.path.join(output_path, f"{sample_folder}_hof.npy"), original)
            for j in range(num_aug):
                np.save(os.path.join(output_path, f"{sample_folder}_aug{j+1}_hof.npy"), augmentations[j])

# Paths
dataset_path = "../dataset_200_3_points"
output_path = "../precomputed_hof_augmented_200"

# Run preprocessing
preprocess_dataset(dataset_path, output_path, num_aug=2)
print("Data augmentation and HOF preprocessing completed!")



Processing videos: 100%|██████████| 320/320 [20:23<00:00,  3.82s/it]

Data augmentation and HOF preprocessing completed!
